In [7]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import numpy as np

file_path = 'C.2_query.tsv'

In [8]:
tf = TriplesFactory.from_path(file_path, delimiter="\t")
training, testing = tf.split([0.8, 0.2], random_state=2025)

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [10531, 2881]


## The Most Basic Model

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

resultTransE = pipeline(
    training=training,
    testing=testing,
    model="TransE",
    model_kwargs=dict(
        embedding_dim=128,
    ),
    training_kwargs=dict(
        num_epochs=50
    ),
    negative_sampler_kwargs=dict(
        num_negs_per_pos=5,
    ),
    random_seed=2025,
    device = device
)

INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/46.0 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/2.88k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.56s seconds


We choose the paper with id: "4ab60d22-67d1-4683-8648-f1f2601d2ce0"

In [15]:
paper = '4ab60d22-67d1-4683-8648-f1f2601d2ce0'

cites = 'cites'
written_by = 'written_by'

entity_embeddings = resultTransE.model.entity_representations[0](indices= None).detach()
relation_embeddings = resultTransE.model.relation_representations[0](indices=None).detach()

citing_paper_id = resultTransE.training.entity_to_id[paper]
cites_id = resultTransE.training.relation_to_id[cites]

citing_paper_embedding = entity_embeddings[citing_paper_id]
cites_embedding = relation_embeddings[cites_id]


In [16]:
# Compute the estimated embedding of the cited paper
cited_paper_embedding = citing_paper_embedding + cites_embedding

# Compute Euclidean distances to all entities
distances = torch.norm(entity_embeddings - cited_paper_embedding.unsqueeze(0), p=2, dim=1)
sorted_distances, sorted_indices = torch.sort(distances)

# Find the closest entity that is not the citing paper itself
top_cited_paper_id = -1
for index in sorted_indices:
    if index != citing_paper_id:
        top_cited_paper_id = index.item()
        break

# Output the result
if top_cited_paper_id != -1:
    print('A likely cited paper was found.')
    print(f'The cited paper is: {resultTransE.training.entity_id_to_label[top_cited_paper_id]}')
    print(f'Most likely cited paper embedding vector:\n{cited_paper_embedding}\n')
else:
    print('No likely cited paper found.')

A likely cited paper was found.
The cited paper is: 4ab4c17b-a9ac-4f37-8e71-5977d5db5a07
Most likely cited paper embedding vector:
tensor([-0.0502, -0.0390, -0.2313,  0.0068,  0.0385,  0.0386, -0.1904,  0.1632,
         0.1959,  0.0702, -0.1881,  0.0619,  0.0166, -0.0050, -0.0968,  0.2545,
        -0.0349, -0.0888,  0.1286, -0.1457,  0.1279, -0.1073,  0.0079,  0.0201,
        -0.0166,  0.1746,  0.1243, -0.0534, -0.0823, -0.0874, -0.0608,  0.0880,
        -0.0561, -0.1020, -0.0227,  0.1750,  0.0651, -0.1019,  0.0642,  0.2905,
         0.0787,  0.1933,  0.0593,  0.0326,  0.0829,  0.0034,  0.2662,  0.0779,
         0.2340, -0.0378,  0.1816,  0.1323,  0.0621, -0.2679, -0.1196, -0.2895,
         0.0332,  0.0253, -0.1010, -0.1703, -0.0118,  0.1329, -0.0556,  0.2032,
        -0.2130, -0.1174, -0.0319,  0.2885,  0.2545,  0.0256,  0.1344, -0.0761,
        -0.3492,  0.1589,  0.1141, -0.0465, -0.0035,  0.0667,  0.2326,  0.2370,
        -0.1149,  0.1648,  0.0026, -0.0030,  0.0944,  0.1727, -0.0902

In [12]:
written_by_id = resultTransE.training.relation_to_id[written_by]
written_by_embedding = relation_embeddings[written_by_id]

author_embedding = cited_paper_embedding + written_by_embedding

print(f'The most likely author embedding vector: \n {author_embedding}')

The most likely author embedding vector: 
 tensor([ 0.0639,  0.1592, -0.2031, -0.1673,  0.1281,  0.0600, -0.0361, -0.0596,
         0.1319,  0.2097, -0.1905,  0.0146, -0.1211,  0.0785, -0.0563,  0.4573,
         0.1426, -0.1085,  0.2079, -0.3335,  0.2982, -0.1708, -0.0671, -0.0128,
        -0.0152,  0.3773,  0.1706,  0.0489, -0.0331, -0.1103,  0.0593, -0.0580,
         0.0245, -0.0605, -0.1389,  0.2875, -0.0016, -0.0091,  0.1372,  0.4293,
         0.1897,  0.3557, -0.0133, -0.0857,  0.1335, -0.0251,  0.1701,  0.0445,
         0.2906, -0.0210,  0.2446,  0.3222,  0.1008, -0.1276, -0.0157, -0.0900,
        -0.1442,  0.1791,  0.0010, -0.0563, -0.0381,  0.0513, -0.0328,  0.3265,
        -0.1433, -0.2202,  0.0280,  0.3171,  0.1404, -0.0057,  0.0169, -0.1635,
        -0.3645, -0.0527, -0.0129, -0.0578,  0.0138,  0.2323,  0.3539,  0.3800,
        -0.0147,  0.0013,  0.1169,  0.1129,  0.0330,  0.3851,  0.1001,  0.1080,
        -0.1248,  0.0763, -0.1977, -0.1474, -0.0483,  0.0197,  0.0080, -0.111

In [13]:
distances = torch.norm(entity_embeddings - author_embedding.unsqueeze(0), p=2, dim=1)  # Euclidean Distance
sorted_distances, sorted_indices = torch.sort(distances)

top_author_id = -1
for index in sorted_indices:
    if index != citing_paper_id:
        top_author_id = index.item()
        break
if top_author_id != -1:
    top_author = resultTransE.training.entity_id_to_label[top_author_id]

    print(f'The most likely author is: {top_author}')

The most likely author is: AUTHOR_331
